# Implementierung eines Reinforcement-Learning-Projekts

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Das Problem
Wir sind Betreiber eine Spedition und sind zuständig für die Belieferung von Supermärkten. Dafür muss Ware zwischen Lagern und den Märkten innerhalb unserer Stadt transportiert werden. Unser innovativer LKW ist selbstfahrend und bekommt lediglich den Auftrag an einer Station (Lager oder Supermarkt) Ware einzusammeln und an einer anderen Station wieder abzuliefern.

## Definitionen
Wir bedienen zwei Lager und zwei Supermärkte.

Lager:
0. A-Lager (A)
1. B-Lager (B)

Supermarkt:
2. C-Markt (C)
3. D-Markt (D)

Diese verteilen sich folgendermaßen in unserer Stadt:

<img src="img/city.png" alt="stadt" width="400"/>

Die Stadt ist eine 6x6-Quadratestadt und mit 36 Positionen, die die Koordinaten $(0,0)$ bis $(5,5)$ haben. Der LKW kann sich frei in der Stadt bewegen, aber nicht durch die Grünstreifen fahren.

Die Anzahl der Zustände ergibt sich folgendermaßen:
* 6 x 6 Positionen
* 4 Orte zu denen die Ware gebracht werden kann (A bis D bzw. 0 bis 3)
* 5 Orte, an denen sich die Ware befindet (A bis D bzw. 0 bis 3 und im LKW (Position Nr.4))

\\[ 6 \cdot 6 \cdot 4 \cdot 5 = 720 \texttt{ mögliche Zustände}\\]

Die Aktionen, die der LKW ausführen kann sind:
0. nach Norden fahren
1. nach Osten fahren
2. nach Süden fahren
3. nach Westen fahren
4. Ware einsammeln
5. Ware abladen

Dabei kann er folgende Belohnungen (und Abzüge) erhalten:
* Ware korrekt abliefern: +20
* Ware falsch einsammeln/abliefern: -10
* Pro Schritt: -1

## Definitionen implementieren

In [ ]:
import copy
import random
import time

Im Folgenden werden die zuvor getroffenen Spezifikationen in ein für den Rechner verständliches Format überführt. Es gibt jeweils sechs Spalten und Reihen (0 bis 5) und eine Menge von Aktionen (ebenfalls 0 bis 5). Für die Ausgabe wird noch eine Überführung in eine Beschreibung vorgenommen. Außerdem werden die Koordinaten der Lager und Supermärlte festgehalten. Die Koordianten haben dabei die Form `(Reihe, Spalte)`.

Die Grünstreifen werden in einer Menge gespeichert. Ein Grünstreifen liegt immer zwischen zwei Feldern. Diese werden in einem Tupel in der Reihenfolge `(Links, Rechts)` bzw. `(Oben, Unten)` angegeben. Daraus ergibt sich für jedes Stück eines Grünstreifens ein Tupel der Form `((Reihe Zelle links, Spalte Zelle links), (Reihe Zelle rechts, Spalte Zelle rechts))` bzw. mit "oben" und "unten", falls es sich um einen horizontalen Streifen handelt.

In [1]:
rows = [row for row in range(0, 6)]
cols = [col for col in range(0, 6)]
actions = {action for action in range(0, 6)}
actions_description = ["Drive north", "Drive east", "Drive south", "Drive west", "Pickup goods", "Dropoff goods"]
stations = [(0,0), (0,3), (5,0), (4,5)]
stations_descriptions = ["Warehouse A", "Warehouse B", "Supermarket C", "Supermarket D"]
position_goods_descriptions = stations_descriptions + ["In the truck"]
walls = {
    ((0,2), (0,3)), #vertikl
    ((1,2), (1,3)),
    ((3,1), (3,2)),
    ((4,0), (4,1)),
    ((5,0), (5,1)),
    ((5,2), (5,3)),
    ((1,0), (2,0)), # horizonal
    ((1,5), (2,5)),
    ((3,4), (4,4)),
    ((3,5), (4,5))
}

Als nächstes soll für jedes Feld im Stadtplan festgehalten werden, welche Aktionen von diesem Feld ausgehend möglich sind. Zunächst wird allen Feldern alle Aktionen zugewiesen. Den am Rand liegenden Feldern wird die Aktion aberkannt, die aus der Stadt heraus führen würde. Einsammel- und Ablieferaktionen sind nur an den zuvor definierten Stationen möglich, weshalb den anderen Feldern diese Aktion ebenfalls entzogen wird. Als nächstes werden Felder betrachtet, die in der direkten Nachbarschaft eines Grünstreifens liegen und dort die Aktionen entfernt, die den LKW dazu veranlassen würden, die Grünanlage zu zerstören. Daraus ergibt sich dann das gewünschte Dictionary mit dem feld als Key und einer Menge möglicher Aktionen als Value.

In [ ]:
possible_actions = dict()

for row in rows:
    for col in cols:
        possible_actions[(row, col)] = copy.deepcopy(actions)
for key in possible_actions:
    (row, col) = key
    # Ränder
    if row == 0:
        possible_actions[key].remove(0)
    elif row == 5:
        possible_actions[key].remove(2)
    if col == 0:
        possible_actions[key].remove(3)
    elif col == 5:
        possible_actions[key].remove(1)
    # Einsammeln, Abliefern
    if (row, col) not in stations:
        possible_actions[key].remove(4)
        possible_actions[key].remove(5)
    # Grünstreifen
    if ((row, col), (row, col + 1)) in walls:
        possible_actions[key].remove(1)
    if ((row, col - 1), (row, col)) in walls:
        possible_actions[key].remove(3)
    if ((row, col), (row + 1, col)) in walls:
        possible_actions[key].remove(2)
    if ((row - 1, col), (row, col)) in walls:
        possible_actions[key].remove(0)

## Die Transport-Klasse

Die Transportklasse speichert den aktuellen Zustand, die grafische Darstellung von diesem sowie den aktuell erreichten Wert. Außerdem speichert sie, wie viele Schritte bereits ausgeführt wurden und wie viele davon falsch bzw. nicht möglich waren. Dies sind Aktionen, die für das Feld, auf dem sich der LKW befindet, nicht in der Menge der möglichen Aktionen sind. Ist die Variable `stepwise` auf True gesetzt, wird zur besseren Sichtbarkeit der Aktionen in der Darstellung nach jeder Aktion eine kurze Pause durchgeführt.

In [ ]:
class Transport():
    def __init__(self, state, stepwise = False):
        self.state = state
        self.canvas = init_canvas(self.state)
        self.current_value = 0
        self.steps = 0
        self.counter_wrong_actions = 0
        self.stepwise = stepwise
        self.pause = True
        self.position_goods_old = self.state[1]
        
        self.canvas[3].on_mouse_down(self.transport_goods)

Die Transition function gibt für eine Aktion den Zustand zurück der folgt, wenn man auf den aktuellen Zustand die gewünschte Aktion anwendet. Falls die Aktion nicht möglich ist, wird der alte Zustand zurückgegeben. Ein Zustand ist dabei folgendermaßen aufgebaut:

`(Position LKW, Position Ware, Position Ziel)`.

Wobei gilt:
* Position LKW = (Reihe LKW, Spalte LKW)
* Position Ware $\in$ \[0, 4\]
* Position Ziel $\in$ \[0, 3\]

Bei der Funktion handelt es sich um eine Funktion der Klasse `Transport`.

In [ ]:
def transition_function(self, action):
    position_truck, position_goods, position_goal = self.state
    if action not in possible_actions[position_truck]:
        return self.state
    
    row, col = position_truck
    if action == 0:
        position_truck = (row - 1, col)
    elif action == 1:
        position_truck = (row, col + 1) 
    elif action == 2:
        position_truck = (row + 1, col)
    elif action == 3:
        position_truck = (row, col - 1)
    elif action == 4:
        if position_goods == stations.index(position_truck):
            position_goods = 4
    elif action == 5:
        if position_goods != 4:
            return self.state
        position_goods = stations.index(position_truck)
            
    return (position_truck, position_goods, position_goal) 
Transport.transition_function = transition_function
del transition_function

`reward_function()` gibt die Belohnung zurück, die furch die Aktion im aktuellen Zustand erzielt wird.
Bei der Funktion handelt es sich ebenfalls um eine Funktion der Klasse `Transport`.

In [ ]:
def reward_function(self, action):
    # Es gibt Abzug auch wenn die Aktion gar nicht möglich ist (beispiel gegen Wand fahren)
    reward = -1 # Pro Schritt
    position_truck, position_goods, position_goal = self.state
    if action not in possible_actions[position_truck]:
        self.counter_wrong_actions += 1
        return reward
    
    if action == 4:
        station_truck = stations.index(position_truck)
        if position_goods != station_truck: # Ware falsch einsammeln
            reward -= 10
    elif action == 5:
        station_truck = stations.index(position_truck)
        if position_goal != station_truck: # Ware falsch abliefern
            reward -= 10
        if position_goal == station_truck: # Ware korrekt abliefern
            reward += 20
            
    return reward
Transport.reward_function = reward_function
del reward_function

`transport_goods()` wird durch Klicken auf die Karte aufgerufen und führt so oft eine zufällige Aktion aus, bis die Ware an ihrem Zielort eingetroffen ist. Nach dem Ausführen wird die Darstellung aktualisiert.

Bei der Funktion handelt es sich auch um eine Funktion der Klasse `Transport`.

In [ ]:
def transport_goods(self, x, y):
    position_truck, position_goods, position_goal = self.state
    while position_goods != position_goal:    
        self.steps += 1
        action = random.randrange(0, 6, 1)
        action_label.value = 'Action: ' + actions_description[action]
        self.current_value += self.reward_function(action)
        self.state = self.transition_function(action)
        position_truck, position_goods, position_goal = self.state
        update_canvas(self)
        if self.stepwise: time.sleep(0.1)
Transport.transport_goods = transport_goods
del transport_goods

Das Notebook, das im Folgenden geladen wird, beinhaltet Funktionen für die grafische Darstellung des aktuellen Zustands.

In [ ]:
%run ./04_implementierung_gui.ipynb

In [ ]:
transport = Transport(((3,2), 2, 1), stepwise = True)
update_canvas(transport)